## Step 1: Check our mapping file for errors¶

The QIIME mapping file contains all of the per-sample metadata, including technical information such as primers and barcodes that were used for each sample, and information about the samples, including what body site they were taken from. In this current tutorial data set we're looking at human microbiome samples from four sites on the bodies of two individuals at mutliple time points. The metadata in this case therefore includes a subject identifier, a timepoint, and a body site for each sample. You can review the map.tsv file to see an example of the data (or view the published [Google Spreadsheet version](https://docs.google.com/spreadsheets/d/1FXHtTmvw1gM4oUMbRdwQIEOZJlhFGeMNUvZmuEFqpps/pubhtml?gid=0&single=true), which is more nicely formatted).

In this step, we run `validate_mapping_file.py` to ensure that our mapping file is compatible with QIIME. First we will use a perfectly normal mapping file to do test this

In [4]:
!validate_mapping_file.py -o ~/qiime_illumina_test/vmf-map/ -m map.tsv

No errors or warnings were found in mapping file.


In this case there were no errors, but if there were we would review the resulting HTML summary to find out what errors are present. You could then fix those in a spreadsheet program or text editor and rerun validate_mapping_file.py on the updated mapping file.

For the sake of illustrating what errors in a mapping file might look like, we've created a bad mapping file (map-bad.tsv). We'll next call validate_mapping_file.py on the file map-bad.tsv. 

In [5]:
!validate_mapping_file.py -o ~/qiime_illumina_test/vmf-map-bad/ -m map-bad.tsv

Errors and/or warnings detected in mapping file.  Please check the log and html file for details.


Let's look at what errors in log file

In [6]:
!cat ~/qiime_illumina_test/vmf-map-bad/map-bad.tsv.log

# Errors and warnings are written as a tab separated columns, with the first column showing the error or warning, and the second column contains the location of the error or warning, written as row,column, where 0,0 is the top left header item (SampleID).  Problems not specific to a particular data cell will be listed as having 'no location'.
Errors -----------------------------
Duplicate barcode AGCTGACTAGTC found.	1,1
Duplicate barcode AGCTGACTAGTC found.	6,1
Warnings ---------------------------
Found invalid character in DaysSinceExperimentStart! header field.	0,9


## Step 2: Demultiplexing and quality filtering sequences

We next need to demultiplex and quality filter our sequences (i.e. assigning barcoded reads to the samples they are derived from). In general, you should get separate fastq files for your sequence and barcode reads. Note that we pass these files while still gzipped to split_libraries_fastq.py command which can handle gzipped or unzipped fastq files. The default strategy in QIIME for quality filtering of Illumina data is described in [Bokulich et al (2013)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3531572/)

In [7]:
!split_libraries_fastq.py -o ~/qiime_illumina_test/slout/ -i forward_reads.fastq.gz -b barcodes.fastq.gz -m map.tsv

The above command creates 3 new files which we can view:

In [8]:
!ls ~/qiime_illumina_test/slout

histograms.txt	seqs.fna  split_library_log.txt


We can examine the results of the split libraries commands by using the less. 

In [9]:
!head ~/qiime_illumina_test/slout/split_library_log.txt

Input file paths
Mapping filepath: map.tsv (md5: f59d06b2c7b7ae60f51b6fa43100ec95)
Sequence read filepath: forward_reads.fastq.gz (md5: 023e0c106f45b2e031eddf1bfc40f83a)
Barcode read filepath: barcodes.fastq.gz (md5: 9bb954658ab8dedc0a560e42019b15f8)

Quality filter results
Total number of input sequences: 302581
Barcode not in mapping file: 37594
Read too short after quality truncation: 76502
Count of N characters exceeds limit: 1074


You can also see how many sequencing reads passed after demultiplexing and quality filtering using the count_seqs.py.

In [10]:
!count_seqs.py -i ~/qiime_illumina_test/slout/seqs.fna


186333  : /home/upendra_35/qiime_illumina_test/slout/seqs.fna (Sequence lengths (mean +/- std): 132.2422 +/- 9.8806)
186333  : Total


## Step 3: OTU picking using an open-reference OTU picking protocol by searching reads against the Greengenes database

Now that we have demultiplexed sequences, we're ready to cluster these sequences into OTUs. The QIIME demo here uses Open-reference OTU picking (though it takes some time but this is currently QIIME's preferred method). Discussion of these methods can be found in Rideout et. al (2014). Note that this command takes the slout/seqs.fna file that was generated in the previous step. Specifically, we set enable_rev_strand_match to True, which allows sequences to match the reference database if either their forward or reverse orientation matches to a reference sequence. This parameter is specified in theparameters file which is passed as -p. You can find information on defining parameters files [here](http://qiime.org/documentation/file_formats.html#qiime-parameters).

This step can take long time to complete since we are working on a small machine. With a larger instance you can take advantage of parallelization. 

In [11]:
!pick_open_reference_otus.py -o ~/qiime_illumina_test/otus/ -i ~/qiime_illumina_test/slout/seqs.fna -p uc_fast_params.txt

The primary output that we get from this command is the OTU table, or the number of times each operational taxonomic unit (OTU) is observed in each sample. QIIME uses the Genomics Standards Consortium Biological Observation Matrix standard (BIOM) format for representing OTU tables. You can find additional information on the BIOM format [here](http://biom-format.org/), and information on converting these files to tab-separated text that can be viewed in spreadsheet programs [here](http://biom-format.org/documentation/biom_conversion.html). Several OTU tables are generated by this command. The one we typically want to work with is otus/otu_table_mc2_w_tax_no_pynast_failures.biom. This has singleton OTUs (or OTUs with a total count of 1) removed, as well as OTUs whose representative (i.e., centroid) sequence couldn't be aligned with [PyNAST](http://bioinformatics.oxfordjournals.org/content/26/2/266.long). It also contains taxonomic assignments for each OTU as observation metadata.

The open-reference OTU picking command also produces a phylogenetic tree where the tips are the OTUs. The file containing the tree is otus/rep_set.tre, and is the file that should be used with otus/otu_table_mc2_w_tax_no_pynast_failures.biom in downstream phylogenetic diversity calculations. The tree is stored in the widely used [newick format](http://scikit-bio.org/docs/latest/generated/skbio.io.newick.html).

To compute some summary statistics of the OTU table we can run the biom summarize-table command.

In [13]:
!biom summarize-table -i ~/qiime_illumina_test/otus/otu_table_mc2_w_tax_no_pynast_failures.biom

Num samples: 34
Num observations: 3426
Total count: 182140
Table density (fraction of non-zero values): 0.122

Counts/sample summary:
 Min: 1114.0
 Max: 11449.0
 Median: 4966.000
 Mean: 5357.059
 Std. dev.: 3333.813
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy

Counts/sample detail:
L3S242: 1114.0
L3S341: 1234.0
L3S360: 1240.0
L3S313: 1304.0
L3S294: 1472.0
L3S378: 1558.0
L2S309: 1842.0
L5S155: 1982.0
L5S240: 2127.0
L5S174: 2176.0
L5S203: 2352.0
L5S104: 2505.0
L5S222: 2771.0
L2S357: 3016.0
L2S222: 4035.0
L2S204: 4131.0
L2S155: 4880.0
L2S382: 5052.0
L2S175: 5437.0
L2S240: 7054.0
L1S257: 7165.0
L6S68: 7281.0
L1S281: 7548.0
L6S20: 7882.0
L1S8: 8320.0
L1S140: 8477.0
L6S93: 8509.0
L1S76: 8837.0
L1S105: 9065.0
L4S112: 9705.0
L1S57: 9793.0
L1S208: 9862.0
L4S63: 10965.0
L4S137: 11449.0


The key piece of information you need to pull from this output is the depth of sequencing (Counts/sample) that should be used in diversity analyses (step 4). Many of the analyses that follow require that there are an equal number of sequences in each sample, so you need to review the output and decide what depth you'd like. Any samples that don't have at least that many sequences will not be included in the analyses, so this is always a trade-off between the number of sequences you throw away and the number of samples you throw away. For some perspective on this, see [Kuczynski 2010](http://www.ncbi.nlm.nih.gov/pubmed/20441597).

## Step 4: Run diversity analyses

Now we run the core_diversity_analyses.py script which does the diversity analyses that users are generally interested in. The main output that users will interact with is the otus/index.html file, which provides links into the different analysis results. Note that in this step we're passing -e (depth of sampling) that should be used for diversity analyses. Based on reviewing the above output from biom summarize-table, 1114 seems to be e value. This value will be study-specific, so don't just use this value on your own data (though it's fine to use that value for this tutorial).

In [14]:
!core_diversity_analyses.py -o ~/qiime_illumina_test/cdout/ -i ~/qiime_illumina_test/otus/otu_table_mc2_w_tax_no_pynast_failures.biom -m map.tsv -t ~/qiime_illumina_test/otus/rep_set.tre -e 1114 

/usr/local/lib/python2.7/dist-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0338088308504 and the largest is 4.5997169389.
  RuntimeWarning
/usr/local/lib/python2.7/dist-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/usr/local/lib/python2.7/dist-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


## Step 5: Run emperor

After this runs, you can reload the Emperor plots that you accessed from the above cdout/index.html links. 

In [15]:
!make_emperor.py -i ~/qiime_illumina_test/cdout/bdiv_even1114/weighted_unifrac_pc.txt -o ~/qiime_illumina_test/cdout/bdiv_even1114/weighted_unifrac_emperor_pcoa_plot -m map.tsv --custom_axes DaysSinceExperimentStart

In [16]:
!make_emperor.py -i ~/qiime_illumina_test/cdout/bdiv_even1114/unweighted_unifrac_pc.txt -o ~/qiime_illumina_test/cdout/bdiv_even1114/unweighted_unifrac_emperor_pcoa_plot -m map.tsv --custom_axes DaysSinceExperimentStart